In [1]:
#step:1 Langchain & dependencies pip install langchain, langchian_community, langchain_google_genai, chroma 
import os 
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import  RetrievalQA

#step:-2 Prepare Documents / Vectorestore 

#Load Text Documents
loader = TextLoader("solar_system.txt")
docs = loader.load()

#split text into chuks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
spilts = text_splitter.split_documents(docs)

#Embeddings 
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#store in vertordb 
vectordb = Chroma.from_documents(spilts, embedding=embeddings)

#Retriever
retriever = vectordb.as_retriever()

#step:-3 Define RAG Tools 
rag_chain = RetrievalQA.from_chain_type(
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0),
    retriever=retriever,
    chain_type="stuff"
)

tools = [
    Tool(
        name="knowledge Base",
        func=rag_chain.run,
        description="use this tools to answer the question from the uploaded documents."
    )
]

#step:-4 Initialize Agent
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

#step:-5 Run the Agent
query ="Summarize the documents for me."
responce = agent.invoke(query)


print("Agent Response:\n", responce)

C:\Users\acer\AppData\Local\Temp\ipykernel_19092\2893324279.py:48: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Action: knowledge Base
Action Input: Summarize the documents.
Observation: The Solar System is a dynamic and vast part of the universe, centered around the Sun, which accounts for about 99.8% of its total mass. It includes eight planets that orbit the Sun in elliptical paths, along with their moons, asteroids, comets, and meteoroids.

Beyond the main planets, the Solar System also contains dwarf planets like Pluto, Ceres, and Eris, which are smaller and haven't cleared their orbits of other debris. Asteroids are rocky remnants primarily found in the asteroid belt between Mars and Jupiter, while comets, composed of ice, dust, and rock, originate from the outer regions and display glowing tails. Meteoroids are small particles from comets or asteroids that become meteors when entering Earth's atmosphere.

Scientists continue to study the Solar System to understand the origins of our universe and the potential for life beyond Earth, highlighting its 